In [6]:
import cv2
import numpy as np


face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")


cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: Could not open camera.")
    exit()


ret, frame1 = cap.read()
if not ret:
    print("Error: Could not read frame.")
    cap.release()
    exit()

prev_gray = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)


MOUTH_MIN_THRESHOLD = 0.8 
MOUTH_MAX_THRESHOLD = 1
FACE_STILL_THRESHOLD = 0.8

mou_count=0
f_count=0
m_count=0
f_flag=0
m_flag=0
mou_flag=0
txt_dur=10

font=cv2.FONT_HERSHEY_SIMPLEX
fontscale=1
thickness=1
color=(0,0,255)



while True:
    ret, frame2 = cap.read()
    if not ret:
        print("Error: Could not read frame.")
        break
    gray_frame = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)


    faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5, minSize=(50, 50))

    for (x, y, w, h) in faces:

        cv2.rectangle(frame2, (x, y), (x + w, y + h), (255, 0, 0), 2)

        face_roi_prev = prev_gray[y:y + h, x:x + w]
        face_roi_next = gray_frame[y:y + h, x:x + w]

        flow = cv2.calcOpticalFlowFarneback(face_roi_prev, face_roi_next, None, 0.5, 3, 15, 3, 5, 1.2, 0)

        magnitude, _ = cv2.cartToPolar(flow[..., 0], flow[..., 1])
        avg_face_motion = np.mean(magnitude)

        face_is_still = avg_face_motion < FACE_STILL_THRESHOLD

        if face_is_still:
           
            mouth_y1 = y + int(2 * h / 3)
            mouth_y2 = y + h
            mouth_x1 = x
            mouth_x2 = x + w

           
            mouth_roi_prev = prev_gray[mouth_y1:mouth_y2, mouth_x1:mouth_x2]
            mouth_roi_next = gray_frame[mouth_y1:mouth_y2, mouth_x1:mouth_x2]

           
            mouth_flow = cv2.calcOpticalFlowFarneback(mouth_roi_prev, mouth_roi_next, None, 0.5, 3, 15, 3, 5, 1.2, 0)

           
            mouth_magnitude, mouth_angle = cv2.cartToPolar(mouth_flow[..., 0], mouth_flow[..., 1])

           
            avg_mouth_motion = np.mean(mouth_magnitude)
            if MOUTH_MIN_THRESHOLD < avg_mouth_motion < MOUTH_MAX_THRESHOLD:
                mou_count=mou_count+1
                print(m_count)

          
            hsv = np.zeros((mouth_roi_prev.shape[0], mouth_roi_prev.shape[1], 3), dtype=np.uint8)
            hsv[..., 1] = 255
            hsv[..., 0] = mouth_angle * 180 / np.pi / 2
            hsv[..., 2] = cv2.normalize(mouth_magnitude, None, 0, 255, cv2.NORM_MINMAX)

            mouth_flow_visual = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

            frame2[mouth_y1:mouth_y2, mouth_x1:mouth_x2] = cv2.addWeighted(
                frame2[mouth_y1:mouth_y2, mouth_x1:mouth_x2], 0.5, mouth_flow_visual, 0.5, 0
            )
    if len(faces)==0:
        f_count+=1
    if len(faces)>1:
        m_count+=1

    if f_count>20:
        f_flag=txt_dur
        f_count=0
    if m_count>20:
        m_flag=txt_dur
        m_count=0
    if mou_count>1:
        mou_flag=txt_dur
        mou_count=0

    if f_flag>0:
        cv2.putText(frame2,"No Faces Detected !!!",(0,30),font,fontscale,color,thickness,cv2.LINE_AA)
        f_flag-=1
    if m_flag>0:
        cv2.putText(frame2,"Multiple Faces Detected !!!",(0,60),font,fontscale,color,thickness,cv2.LINE_AA)
        m_flag-=1
    if mou_flag>0:
        cv2.putText(frame2,"Talking Detected !!!",(0,90),font,fontscale,color,thickness,cv2.LINE_AA)
        mou_flag-=1
        

    cv2.imshow('Face and Mouth Motion Detection', frame2)


    prev_gray = gray_frame


    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Error: Could not open camera.
Error: Could not read frame.


[ WARN:0@1673.946] global cap_v4l.cpp:999 open VIDEOIO(V4L2:/dev/video0): can't open camera by index
[ERROR:0@1673.947] global obsensor_uvc_stream_channel.cpp:158 getStreamChannelGroup Camera index out of range


error: OpenCV(4.10.0) /io/opencv/modules/imgproc/src/color.cpp:196: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'
